In [8]:
import numpy as np
import pandas as pd
import os

cur_dir = os.getcwd() 
print(cur_dir)
admissions = pd.read_csv(cur_dir + '/binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
print(data.head())
data = data.drop('rank', axis=1)
print(data.head())

/Users/SKIM/JupyterNotebook/Udacity-ML-Google/gradient-decent
   admit  gre   gpa  rank  rank_1  rank_2  rank_3  rank_4
0      0  380  3.61     3     0.0     0.0     1.0     0.0
1      1  660  3.67     3     0.0     0.0     1.0     0.0
2      1  800  4.00     1     1.0     0.0     0.0     0.0
3      1  640  3.19     4     0.0     0.0     0.0     1.0
4      0  520  2.93     4     0.0     0.0     0.0     1.0
   admit  gre   gpa  rank_1  rank_2  rank_3  rank_4
0      0  380  3.61     0.0     0.0     1.0     0.0
1      1  660  3.67     0.0     0.0     1.0     0.0
2      1  800  4.00     1.0     0.0     0.0     0.0
3      1  640  3.19     0.0     0.0     0.0     1.0
4      0  520  2.93     0.0     0.0     0.0     1.0


In [11]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
print(data.head())
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)
print(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

     admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
154      0  0.244347  0.819196     0.0     0.0     1.0     0.0
76       0 -0.268624 -0.051032     0.0     0.0     1.0     0.0
341      1 -0.268624 -1.923339     0.0     0.0     1.0     0.0
370      1 -0.439614  1.030160     0.0     1.0     0.0     0.0
223      0  1.783262  0.239044     0.0     0.0     1.0     0.0
[329 326  66  24 205 105 221  60 183 340 120  59 262 378 190   7 203 231
 362 298 127 286  39 371 131  17 230 278 369 166 124  47 389 290 125  65
 100 165 198  27 158 135 280  91 237  70 367 244  79 211 336 142 172 246
 236 224  32 229 370 254 192 164  78 384 261  53 292 288 312  73 117 101
  56 282 291 128 289 251 137 114 126  31 285 335 320 358 350 226 108  42
 296 272 390 255 139 210  97 283 239  94 373 233 398   3 136 394 256 163
 383  92 307 247 153 134 201 242 352  41 342 215 376 351  86 202  26 220
 284 175 259  57  88  98 302 159 238  40 138  96 331 379 218  74  68 118
  61 337 250 152 180 116 107 268 385  

In [8]:
import numpy as np
from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1/n_features**.5, size=n_features)

# Neural Network parameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Activation of the output unit
        output = sigmoid(np.dot(x, weights))

        # The error, the target minues the network output
        error = y - output

        # The gradient descent step, the error times the gradient times the inputs
        del_w += error * output * (1 - output) * x

        # Update the weights here. The learning rate times the 
        # change in weights, divided by the number of records to average
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

('Train loss: ', 0.26276093849966364)
('Train loss: ', 0.20928619409324895)
('Train loss: ', 0.20084292908073417)
('Train loss: ', 0.19862156475527884)
('Train loss: ', 0.19779851396686018)
('Train loss: ', 0.19742577912189863)
('Train loss: ', 0.19723507746241065)
('Train loss: ', 0.19712945625092465)
('Train loss: ', 0.19706766341315074)
('Train loss: ', 0.19703005801777368)
Prediction accuracy: 0.725
